# Tox21 reformat data

Reformat Tox21 data for export and visualization ( _e.g._ in [DataWarrior](http://www.openmolecules.org/datawarrior/) ).

In [1]:
%run setup.py

In [2]:
from math import log10

from functools import partial

### Config

In [3]:
# Endpoint columns (ordered)...

endpoint_cols = ['Summary', 'Activity', 'Potency (uM)', 'Efficacy (%)', 'Viability Activity', 'Viability Potency (uM)', 'Viability Efficacy (%)']

In [4]:
# Categories for the hit-call enpoints...

summary_categories =  ['active agonist', 'inconclusive agonist', 'inconclusive agonist (cytotoxic)', 'inconclusive agonist (fluorescent)', 'active antagonist', 'inconclusive antagonist', 'inconclusive antagonist (cytotoxic)', 'inconclusive', 'inactive']

activity_categories = ['active agonist', 'inconclusive agonist',                                                                           'active antagonist', 'inconclusive antagonist',                                        'inconclusive', 'inactive']

In [5]:
# Directory for reading and writing data files...

data_dir = 'data'

In [6]:
tick, cross = '\u2713', '\u2717'

### Initialisation

In [7]:
if not 'logger' in locals(): logger = make_logger.run(__name__)

In [8]:
# Utility function to reorder dataframe columns...

def insert_column_after(df, col0, col1):
    cols = df.columns.tolist()
    cols.remove(col1)
    cols.insert(cols.index(col0)+1, col1)
    return df[cols]

### Reload Tox21 data

See [here](4_Tox21_get_data.ipynb) for details of the data-retrieval procedure.

In [9]:
tox21_data_df = pd.read_pickle(os.path.join(data_dir, 'tox21_data.pkl'))

tox21_data_df.shape

(1694115, 5)

In [10]:
tox21_data_df.head(1)

,AID,SID,endpoint,value,dtype
0,743228,144206330,Summary,inactive,sval


### Reformat data

Pivot data table to create columns for the individual endpoints...

In [11]:
index_cols = ['AID', 'SID']

pivoted_df = (
    tox21_data_df.pivot_table(index=index_cols, columns=['endpoint'], values=['value'], aggfunc='first')
     .reset_index()
     .fillna('')
)

pivoted_df.shape

(346670, 9)

In [12]:
pivoted_df.head(1)

AID        SID     value                                                                                                     
endpoint                     Activity Efficacy (%) Potency (uM)   Summary Viability Activity Viability Efficacy (%) Viability Potency (uM)
0         720516  144203552  inactive            0               inactive           inactive                      0

In [13]:
# Reformat column headers...

n_index_cols = len(index_cols)

pivoted_df.columns = pivoted_df.columns.droplevel(1).tolist()[:n_index_cols] + pivoted_df.columns.droplevel(0).tolist()[n_index_cols:]

# Reorder columns...

pivoted_df = pivoted_df[index_cols + endpoint_cols]

In [14]:
pivoted_df.head(1)

,AID,SID,Summary,Activity,Potency (uM),Efficacy (%),Viability Activity,Viability Potency (uM),Viability Efficacy (%)
0,720516,144203552,inactive,inactive,,0,inactive,,0


###  Column types

Modify column types where necessary.

In [15]:
pivoted_df.dtypes

AID                        int64
SID                        int64
Summary                   object
Activity                  object
Potency (uM)              object
Efficacy (%)              object
Viability Activity        object
Viability Potency (uM)    object
Viability Efficacy (%)    object
dtype: object

#### Numeric columns

Convert numeric ( _i.e._ the Potency and Efficacy ) columns from object to float...

In [16]:
def to_float(x):
    
    try:
        
        return np.float64(x)
        
    except ValueError as exception:
        
        return np.nan
    
for col in pivoted_df.columns[pivoted_df.columns.str.contains('|'.join(re.escape(x) for x in ['Potency (uM)', 'Efficacy (%)']))]:
    
    pivoted_df[col] = pivoted_df[col].apply(to_float)

#### Hit calls

Set the various hit-call columns to be ordered categorical variables, so that their (non-alphabetical) ordering is preseved...

In [17]:
pivoted_df['Summary'] = pivoted_df['Summary'].astype('category', categories=summary_categories, ordered=True)

for col in pivoted_df.columns[pivoted_df.columns.str.contains('Activity')]:
    
    pivoted_df[col] = pivoted_df[col].astype('category', categories=activity_categories, ordered=True)

In [18]:
pivoted_df.dtypes

AID                          int64
SID                          int64
Summary                   category
Activity                  category
Potency (uM)               float64
Efficacy (%)               float64
Viability Activity        category
Viability Potency (uM)     float64
Viability Efficacy (%)     float64
dtype: object

### Data modification

Some data modification to enable easier visualization in DataWarrior.

In [19]:
# Add AC50 columns for the dose-response (i.e. 'Potency') columns...

potency_to_ac50 = lambda x: -log10(x/10**6)

pattern = re.escape('Potency (uM)')

for potency_col in pivoted_df.columns[pivoted_df.columns.str.contains(pattern)]:
    
    ac50_col = re.sub(pattern, 'AC50', potency_col)
        
    pivoted_df[ac50_col] = pivoted_df[potency_col].apply(potency_to_ac50)
    
    pivoted_df = insert_column_after(pivoted_df, potency_col, ac50_col)
    
pivoted_df.shape

(346670, 11)

In [20]:
pivoted_df.head(1)

,AID,SID,Summary,Activity,Potency (uM),AC50,Efficacy (%),Viability Activity,Viability Potency (uM),Viability AC50,Viability Efficacy (%)
0,720516,144203552,inactive,inactive,NaN,NaN,0.0,inactive,NaN,NaN,0.0


In [21]:
# There are some excessively high and low Efficacy values, which can make visualization difficult...

max_efficacy, min_efficacy = 200.0, -200.0

efficacy_cols = pivoted_df.columns[pivoted_df.columns.str.contains(re.escape('Efficacy (%)'))]

for col in efficacy_cols:
    
    print("{:<25s}: {:4d} HIGH, {:4d} LOW".format(col, (pivoted_df[col] > max_efficacy).sum(), (pivoted_df[col] < min_efficacy).sum()))

Efficacy (%)             : 1065 HIGH,   23 LOW
Viability Efficacy (%)   :  190 HIGH,    1 LOW


In [22]:
# Trim the excessive Efficacy values..

trimmer = lambda x: partial(max, min_efficacy)(partial(min, max_efficacy)(x))

for col in efficacy_cols:
    
    pivoted_df[col] = pivoted_df[col].apply(trimmer)
    
for col in efficacy_cols:
    
    print("{:<25s}: {:4d} HIGH, {:4d} LOW".format(col, (pivoted_df[col] > max_efficacy).sum(), (pivoted_df[col] < min_efficacy).sum()))

Efficacy (%)             :    0 HIGH,    0 LOW
Viability Efficacy (%)   :    0 HIGH,    0 LOW


In [23]:
# Prepend category code (a number) to string value of categorical variables to preserve ordering one exported as string...

hit_call_cols = ['Summary', 'Activity', 'Viability Activity']

for col in hit_call_cols:
    
    pivoted_df[col] = ["{}) {}".format(x, y) if type(y) == str else '' for x, y in zip(pivoted_df[col].cat.codes, pivoted_df[col])]

In [24]:
pivoted_df.head(1)

,AID,SID,Summary,Activity,Potency (uM),AC50,Efficacy (%),Viability Activity,Viability Potency (uM),Viability AC50,Viability Efficacy (%)
0,720516,144203552,8) inactive,5) inactive,NaN,NaN,0.0,5) inactive,NaN,NaN,0.0


### Compound structures

Add compound structures (as SMILES).

See [here](1_Tox21_compounds.ipynb) for further details on the Tox21 compounds.

In [25]:
# Load SID-to-structure mapping table (keeping only required colummns)...

sid_mols_df = pd.read_pickle(os.path.join(data_dir, 'sid_mols.pkl')).drop(['inchi', 'mol'], axis=1)

sid_mols_df.shape

(14252, 5)

In [26]:
sid_mols_df.head(1)

,SID,CID,parent_CID,smiles,inchikey
0,312345403,6436006,5365247,CN(C)C/C=C(/C1=CC=C(C=C1)Br)\C2=CN=CC=C2,OYPPVKRFBIWMSX-SXGWCWSVSA-N


Note that a small number of SIDs (187) do not have structures, and hence do not appear in the structure table 'sid_mols_df'...

In [27]:
df = pivoted_df.merge(sid_mols_df, on='SID')

df[df['inchikey'].isnull()]['SID'].unique().size

187

The pivoted data is thus joined to the structure table using an inner join; records without an associated structure are lost, but as there is no structure it is unlikely to be missed.

In [28]:
pivoted_df = pivoted_df.merge(sid_mols_df, on='SID', how='inner')

pivoted_df.shape

(341566, 15)

In [29]:
pivoted_df.head(1)

,AID,SID,Summary,Activity,Potency (uM),AC50,Efficacy (%),Viability Activity,Viability Potency (uM),Viability AC50,Viability Efficacy (%),CID,parent_CID,smiles,inchikey
0,720516,144203552,8) inactive,5) inactive,NaN,NaN,0.0,5) inactive,NaN,NaN,0.0,12850184,5460352,C(C(=O)[C@H]([C@@H]([C@H](C(=O)O)O)O)O)O,IZSRJDGCGRAUAR-MROZADKFSA-N


### Case study info

Add case study information.

See [here](Tox21_PubChem_vs_case_study_compounds.ipynb) for further details on the modified version of the case study compounds table.

In [30]:
# Load the modified version of the case study compounds table...

cs_cmpds_df = pd.read_pickle(os.path.join(data_dir, 'case_study_compounds_2.pkl'))

cs_cmpds_df.shape

(176, 9)

In [31]:
cs_cmpds_df.head(1)

,inchikey,name,Case Study 1,Case Study 2,Case Study 3,Case Study 4,Case Study 5,Case Study 6,Case Study 7
0,ZRYCZAWRXHAAPZ-UHFFFAOYSA-N,"2,2-dimethyl-pentanoic acid",✗,✓,✗,✗,✗,✗,✗


In [32]:
# Add an 'Any Case Study' column for convenience...

cs_cmpds_df['Any Case Study'] = tick

cs_cmpds_df = insert_column_after(cs_cmpds_df, 'name', 'Any Case Study')

In [33]:
pivoted_df = pivoted_df.merge(cs_cmpds_df, on='inchikey', how='left')

pivoted_df.shape

(341566, 24)

In [34]:
# Make the cases study compound name column less ambiguous...

pivoted_df.rename(columns={'name': 'Compound Name'}, inplace=True)

In [35]:
# Fix missing values introduced by joins...

pivoted_df['Compound Name'].fillna('', inplace=True)

for col in pivoted_df.columns[pivoted_df.columns.str.match('^(?:Any Case Study|Case Study \d+)$')]:
    
    pivoted_df[col].fillna(cross, inplace=True)    

In [36]:
# Add a numeric version of 'Any Case Study' to allow sizing by case study membership in DataWarrior...

pivoted_df['ACS'] = pivoted_df['Any Case Study'].apply(lambda x: 1 if x == tick else 0)

In [37]:
pivoted_df.head(1)

,AID,SID,Summary,Activity,Potency (uM),AC50,Efficacy (%),Viability Activity,Viability Potency (uM),Viability AC50,Viability Efficacy (%),CID,parent_CID,smiles,inchikey,Compound Name,Any Case Study,Case Study 1,Case Study 2,Case Study 3,Case Study 4,Case Study 5,Case Study 6,Case Study 7,ACS
0,720516,144203552,8) inactive,5) inactive,NaN,NaN,0.0,5) inactive,NaN,NaN,0.0,12850184,5460352,C(C(=O)[C@H]([C@@H]([C@H](C(=O)O)O)O)O)O,IZSRJDGCGRAUAR-MROZADKFSA-N,,✗,✗,✗,✗,✗,✗,✗,✗,0


### Assay information

See [here](0_Tox21_assays.ipynb) for further details on the Tox21 summary assays.

In [38]:
summary_assays_df = pd.read_pickle(os.path.join(data_dir, 'tox21_summary_assays.pkl'))

summary_assays_df.shape

(35, 4)

In [39]:
summary_assays_df.head(1)

,AID,assay_name,target,gene_id
0,720516,qHTS assay for small molecules that induce genotoxicity in human embryonic kidney cells expressing luciferase-tagged ATAD5,ATPase family AAA domain-containing protein 5; Chromosome fragility-associated gene 1 protein,296439460


In [40]:
pivoted_df = pivoted_df.merge(summary_assays_df, on='AID')

pivoted_df.shape

(341566, 28)

In [41]:
# Prettify column names...

pivoted_df.rename(columns={'assay_name': 'Assay Name', 'target': 'Target', 'gene_id': 'Gene ID'}, inplace=True)

# reorder columns...

pivoted_df = insert_column_after(pivoted_df, 'AID', 'Assay Name')

In [42]:
pivoted_df.head(1)

,AID,Assay Name,SID,Summary,Activity,Potency (uM),AC50,Efficacy (%),Viability Activity,Viability Potency (uM),Viability AC50,Viability Efficacy (%),CID,parent_CID,smiles,inchikey,Compound Name,Any Case Study,Case Study 1,Case Study 2,Case Study 3,Case Study 4,Case Study 5,Case Study 6,Case Study 7,ACS,Target,Gene ID
0,720516,qHTS assay for small molecules that induce genotoxicity in human embryonic kidney cells expressing luciferase-tagged ATAD5,144203552,8) inactive,5) inactive,NaN,NaN,0.0,5) inactive,NaN,NaN,0.0,12850184,5460352,C(C(=O)[C@H]([C@@H]([C@H](C(=O)O)O)O)O)O,IZSRJDGCGRAUAR-MROZADKFSA-N,,✗,✗,✗,✗,✗,✗,✗,✗,0,ATPase family AAA domain-containing protein 5; Chromosome fragility-associated gene 1 protein,296439460


In [43]:
# Sort first on assay name, as this is likely how the data will be browsed...

pivoted_df.sort_values(['Assay Name', 'SID'], inplace=True)

### Export data 

Export data as CSV file for import into a visualization application such as DataWarrior.

In [44]:
pivoted_df.to_csv(os.path.join(data_dir, 'tox21_data_for_visualization.csv'), index=False)

### Examining hit-call counts

A brief digression to look at the distribution of the various hit calls.

Independent counts for various hit calls...

In [45]:
pivoted_df['Summary'].value_counts().to_frame('N').reset_index(drop=False).rename(columns={'index': 'Summary'}) # .sort_values('Summary')

,Summary,N
0,8) inactive,273905
1,7) inconclusive,16899
2,0) active agonist,13266
3,1) inconclusive agonist,11539
4,6) inconclusive antagonist (cytotoxic),10169
5,4) active antagonist,8361
6,5) inconclusive antagonist,6500
7,2) inconclusive agonist (cytotoxic),540
8,3) inconclusive agonist (fluorescent),387


In [46]:
pivoted_df['Activity'].value_counts().to_frame('N').reset_index(drop=False).rename(columns={'index': 'Activity'}) # .sort_values('Activity')

,Activity,N
0,5) inactive,273905
1,2) active antagonist,18803
2,0) active agonist,17253
3,1) inconclusive agonist,16462
4,3) inconclusive antagonist,13189
5,4) inconclusive,1954


In [47]:
pivoted_df['Viability Activity'].value_counts().to_frame('N').reset_index(drop=False).rename(columns={'index': 'Viability Activity'}) # .sort_values('Viability Activity')

,Viability Activity,N
0,5) inactive,243070
1,,50871
2,2) active antagonist,21828
3,3) inconclusive antagonist,17489
4,1) inconclusive agonist,6226
5,4) inconclusive,1288
6,0) active agonist,794


Contingent counts for various hit calls...

In [48]:
df = pivoted_df[hit_call_cols].copy()

df['count'] = '' # NB Add dummy column for group-by count

df = df.groupby(hit_call_cols).count().dropna() # NB Use of categorical variables introduces empty cells

df['count'] = df['count'].astype(int) # Introduction of empty cells coerces column to float

df

count
Summary                                Activity                   Viability Activity                
0) active agonist                      0) active agonist                                        1793
                                                                  0) active agonist              144
                                                                  1) inconclusive agonist        230
                                                                  2) active antagonist          1723
                                                                  3) inconclusive antagonist     995
                                                                  4) inconclusive                 51
                                                                  5) inactive                   8330
1) inconclusive agonist                0) active agonist          0) active agonist                1
                                                                  1) inconclusive agonist          3
                                                                  2) active antagonist            27
                                                                  3) inconclusive antagonist      16
                                                                  4) inconclusive                  1
                                                                  5) inactive                     93
                                       1) inconclusive agonist                                  1018
                                                                  0) active agonist               69
                                                                  1) inconclusive agonist        286
                                                                  2) active antagonist           634
                                                                  3) inconclusive antagonist     573
                                                                  4) inconclusive                 40
                                                                  5) inactive                   8778
2) inconclusive agonist (cytotoxic)    0) active agonist          2) active antagonist           143
                                                                  3) inconclusive antagonist      74
                                       1) inconclusive agonist    2) active antagonist           188
                                                                  3) inconclusive antagonist     135
3) inconclusive agonist (fluorescent)  0) active agonist                                         101
                                                                  1) inconclusive agonist          5
                                                                  2) active antagonist            24
                                                                  3) inconclusive antagonist      21
                                                                  4) inconclusive                  3
                                                                  5) inactive                    190
                                       1) inconclusive agonist                                    10
                                                                  1) inconclusive agonist          3
                                                                  2) active antagonist             3
                                                                  3) inconclusive antagonist       5
                                                                  5) inactive                     22
4) active antagonist                   2) active antagonist                                      210
                                                                  0) active agonist              120
                                                                  1) inconclusive agonist        254
                                                       

In [49]:
# Alternative view of contingent counts...

hit_call_cols_2 = list(reversed(hit_call_cols))

df = pivoted_df[hit_call_cols_2].copy()

df['count'] = '' # NB Add dummy column for group-by count

df = df.groupby(hit_call_cols_2).count().dropna() # NB Use of categorical variables introduces empty cells

df['count'] = df['count'].astype(int) # Introduction of empty cells coerces column to float

df

count
Viability Activity         Activity                   Summary                                       
                           0) active agonist          0) active agonist                         1793
                                                      3) inconclusive agonist (fluorescent)      101
                                                      7) inconclusive                            693
                           1) inconclusive agonist    1) inconclusive agonist                   1018
                                                      3) inconclusive agonist (fluorescent)       10
                                                      7) inconclusive                            878
                           2) active antagonist       4) active antagonist                       210
                                                      7) inconclusive                            701
                           3) inconclusive antagonist 5) inconclusive antagonist                 364
                                                      7) inconclusive                            925
                           4) inconclusive            7) inconclusive                            223
                           5) inactive                8) inactive                              43955
0) active agonist          0) active agonist          0) active agonist                          144
                                                      1) inconclusive agonist                      1
                                                      7) inconclusive                              5
                           1) inconclusive agonist    1) inconclusive agonist                     69
                                                      7) inconclusive                             12
                           2) active antagonist       4) active antagonist                       120
                           3) inconclusive antagonist 5) inconclusive antagonist                  43
                                                      7) inconclusive                              2
                           4) inconclusive            7) inconclusive                              5
                           5) inactive                8) inactive                                393
1) inconclusive agonist    0) active agonist          0) active agonist                          230
                                                      1) inconclusive agonist                      3
                                                      3) inconclusive agonist (fluorescent)        5
                                                      7) inconclusive                             17
                           1) inconclusive agonist    1) inconclusive agonist                    286
                                                      3) inconclusive agonist (fluorescent)        3
                                                      7) inconclusive                             87
                           2) active antagonist       4) active antagonist                       254
                                                      7) inconclusive                             20
                           3) inconclusive antagonist 5) inconclusive antagonist                 169
                                                      7) inconclusive                             54
                           4) inconclusive            7) inconclusive                             54
                           5) inactive                8) inactive                               5044
2) active antagonist       0) active agonist          0) active agonist                         1723
                                                      1) inconclusive agonist                     27
                                                      2) inconclusive agonist (cytotoxic)        143
                                                      3